# GPT Action Library: Jira

このライブラリは、JiraとGPTを統合するためのアクションを提供します。Jiraは、プロジェクト管理、課題追跡、およびアジャイル開発ワークフローのための人気のあるツールです。

## 主な機能

- 課題の作成、更新、検索
- プロジェクトとボードの管理
- コメントとワークフローの操作
- レポートとダッシュボードの生成

## 前提条件

- Jiraインスタンスへのアクセス権限
- 適切なAPIトークンまたは認証情報
- 必要なJiraプロジェクトの権限

## セットアップ

1. Jira APIトークンを生成
2. 認証情報を設定
3. 必要なスコープと権限を確認
4. GPTアクションを設定

このライブラリを使用することで、GPTを通じてJiraワークフローを効率的に管理できます。

## はじめに

このページでは、特定のアプリケーション向けのGPT Actionを構築する開発者向けの手順とガイドを提供します。進める前に、まず以下の情報をよく理解しておいてください：

- [GPT Actionsの概要](https://platform.openai.com/docs/actions)
- [GPT Actions Libraryの概要](https://platform.openai.com/docs/actions/actions-library)
- [GPT Actionをゼロから構築する例](https://platform.openai.com/docs/actions/getting-started)

この特定のGPT Actionは、Atlassianのプロジェクトおよびチケット管理ツールであるJiraに接続する方法の概要を提供します。このアクションはユーザーのコンテキストを前提とし、指定されたプロジェクト内の課題を読み書きできるようにします。

### 価値 + ビジネス活用事例

**価値**: ユーザーはChatGPTの自然言語機能を活用して、Jira Cloudに直接接続できるようになりました

**使用例**: 
- ユーザーは特定のプロジェクトの最新の課題を読み込み、ChatGPTを使用して解決策を提供してもらうことができます
- ユーザーはChatGPTに指示することで、課題やサブタスクを作成・変更し、特定のユーザーに割り当てることができます

## アプリケーション情報

### アプリケーションキーリンク

開始する前に、アプリケーションの以下のリンクを確認してください：
- アプリケーションWebサイト: https://<YOUR_SUBDOMAIN>.atlassian.net/jira
- アプリケーションAPI ドキュメント: https://developer.atlassian.com/cloud/jira/platform/rest/v3/intro/
- アプリケーションOAuth 2.0 ドキュメント: https://developer.atlassian.com/cloud/jira/platform/oauth-2-3lo-apps/

### アプリケーションの前提条件

開始する前に、アプリケーション環境で以下の手順を実行していることを確認してください：
- [Atlassian Cloud Developer Console](https://developer.atlassian.com/console/myapps/)でアプリケーションを作成するためのアクセス権限があることを確認する

## ChatGPT ステップ

### カスタムGPT指示

Custom GPTを作成したら、以下のテキストをInstructionsパネルにコピーしてください。質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

In [ ]:
**Context**: you are specialized GPT designed to create and edit issues through API connections to Jira Cloud. This GPT can create, read, and edit project issues based on user instructions.

**Instructions**:
- When asked to perform a task, use the available actions via the api.atlassian.com API.
- When asked to create an issue, use the user's input to synthesize a summary and description and file the issue in JIRA.
- When asked to create a subtask, assume the project key and parent issue key of the currently discussed issue. Clarify with if this context is not available.
- When asked to assign an issue or task to the user, first use jql to query the current user's profile and use this account as the assignee. 
- Ask for clarification when needed to ensure accuracy and completeness in fulfilling user requests.

### OpenAPI スキーマ

Custom GPTを作成したら、以下のテキストをActionsパネルにコピーしてください。質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

*注意: URLの中のプレースホルダー<CLOUD_ID>を、あなたのクラウド環境の一意のIDに置き換えてください。この値は https://<YOUR_SUBDOMAIN>.atlassian.net/_edge/tenant_info にアクセスすることで確認できます*

In [ ]:
openapi: 3.1.0
info:
  title: Jira API
  description: API for interacting with Jira issues and sub-tasks.
  version: 1.0.0
servers:
  - url: https://api.atlassian.com/ex/jira/<CLOUD_ID>/rest/api/3
    description: Jira Cloud API
components:
  securitySchemes:
    OAuth2:
      type: oauth2
      flows:
        authorizationCode:
          authorizationUrl: https://auth.atlassian.com/authorize
          tokenUrl: https://auth.atlassian.com/oauth/token
          scopes:
            read:jira-user: Read Jira user information
            read:jira-work: Read Jira work data
            write:jira-work: Write Jira work data
  schemas:
    Issue:
      type: object
      properties:
        id:
          type: string
        key:
          type: string
        fields:
          type: object
          properties:
            summary:
              type: string
            description:
              type: string
            issuetype:
              type: object
              properties:
                name:
                  type: string
paths:
  /search:
    get:
      operationId: getIssues
      summary: Retrieve a list of issues
      parameters:
        - name: jql
          in: query
          required: false
          schema:
            type: string
        - name: startAt
          in: query
          required: false
          schema:
            type: integer
        - name: maxResults
          in: query
          required: false
          schema:
            type: integer
      responses:
        '200':
          description: A list of issues
          content:
            application/json:
              schema:
                type: object
                properties:
                  issues:
                    type: array
                    items:
                      $ref: '#/components/schemas/Issue'
  /issue:
    post:
      operationId: createIssue
      summary: Create a new issue
      requestBody:
        required: true
        content:
          application/json:
            schema:
              type: object
              properties:
                fields:
                  type: object
                  properties:
                    project:
                      type: object
                      properties:
                        key:
                          type: string
                    summary:
                      type: string
                    description:
                      type: string
                    issuetype:
                      type: object
                      properties:
                        name:
                          type: string
      responses:
        '201':
          description: Issue created successfully
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/Issue'
  /issue/{issueIdOrKey}:
    get:
      operationId: getIssue
      summary: Retrieve a specific issue
      parameters:
        - name: issueIdOrKey
          in: path
          required: true
          schema:
            type: string
      responses:
        '200':
          description: Issue details
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/Issue'
    put:
      operationId: updateIssue
      summary: Update an existing issue
      parameters:
        - name: issueIdOrKey
          in: path
          required: true
          schema:
            type: string
      requestBody:
        required: true
        content:
          application/json:
            schema:
              type: object
              properties:
                fields:
                  type: object
                  properties:
                    summary:
                      type: string
                    description:
                      type: string
                    issuetype:
                      type: object
                      properties:
                        name:
                          type: string
      responses:
        '204':
          description: Issue updated successfully
  /issue:
    post:
      operationId: createSubTask
      summary: Create a sub-task for an issue
      requestBody:
        required: true
        content:
          application/json:
            schema:
              type: object
              properties:
                fields:
                  type: object
                  properties:
                    project:
                      type: object
                      properties:
                        key:
                          type: string
                    parent:
                      type: object
                      properties:
                        key:
                          type: string
                    summary:
                      type: string
                    description:
                      type: string
                    issuetype:
                      type: object
                      properties:
                        name:
                          type: string
      responses:
        '201':
          description: Sub-task created successfully
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/Issue'
security:
  - OAuth2:
      - read:jira-user
      - read:jira-work
      - write:jira-work


## 認証手順

以下は、Jiraでの認証設定に関する手順です。ご質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

### Jiraステップ

1. <b>アプリケーションの作成</b>: 最初のステップは、ChatGPTとの統合のためにJiraで新しいアプリケーションを作成することです。これは[Atlassian Developer Console](https://developer.atlassian.com/console/myapps/)にアクセスし、**Create**をクリックして**OAuth 2.0 Integration**を選択することで実行できます。

![gptactions_jira_devconsole.png](../../../images/gptactions_jira_devconsole.png)

ここから、インテグレーションの名前を入力し、**作成**をクリックするだけです。

![gptactions_jira_newapplication.png](../../../images/gptactions_jira_newapplication.png)

2. <b>権限の定義</b>: 次に、アプリケーションに必要な権限を提供する必要があります。新しいアプリケーション内で、サイドバーから**Permissions**メニューを開き、**Jira API**を見つけて**Add**をクリックし、その後**Configure**をクリックします。

![gptactions_jira_permissions.png](../../../images/gptactions_jira_permissions.png)

必要な権限は、GPTの意図された機能によって異なります。このシナリオでは、Jiraの課題を読み書きしたいので、**Jira platform REST API**の下で**Edit Scopes**をクリックして、以下のスコープを選択します：

- read:jira-work
- write:jira-work
- read:jira-user

選択したら、**Save**をクリックします

![gptactions_jira_scopes.png](../../../images/gptactions_jira_scopes.png)

3. <b>プレースホルダーコールバックURLの設定</b>: 次のステップを完了し、ChatGPTとJira間の安全な認証を有効にするための**Client ID**と**Secret**を取得するために、まずプレースホルダーコールバックURLを追加する必要があります。これは、サイドバーの**Authorization**をクリックし、**OAuth 2.0 (3LO)**の横にある**Configure**をクリックすることで実現できます。ここで単純にプレースホルダーURLを入力し、**Save Changes**をクリックします。

![gptactions_jira_placeholder.png](../../../images/gptactions_jira_placeholder.png)

4. <b>アプリケーションクライアントID/シークレット</b>: 次のステップは、ChatGPTとJira間の安全な認証を有効にするための**クライアントID**と**シークレット**を見つけることです。これらの値は、サイドバーの**設定**をクリックし、**認証詳細**まで下にスクロールすることで確認できます。

      設定の次の段階でこれらの値が必要になるため、このページは開いたままにしておいてください！

![gptactions_jira_clientsecret.png](../../../images/gptactions_jira_clientsecret.png)

### ChatGPTにおいて

ChatGPTで、「Authentication」をクリックし、**「OAuth」**を選択してください。以下の情報を入力してください。

- **Client ID**: Jira設定の**ステップ3**で取得した**Client ID**
- **Client Secret**: Jira設定の**ステップ3**で取得した**Secret**
- **Authorization URL**: https://auth.atlassian.com/authorize
- **Token URL**: https://auth.atlassian.com/oauth/token
- **Scope**: read:jira-work write:jira-work read:jira-user 
- **Token Exchange Method**: Default (POST Request)

### アクション後のステップ

ChatGPTで認証を設定したら、アプリケーション内で以下の手順に従ってActionを完成させてください。

- GPT ActionからコールバックURLをコピーする

![gptactions_jira_redirect.png](../../../images/gptactions_jira_redirect.png)

- Atlassian Developer Consoleのアプリケーションで、**Authorization**サイドバータブに移動し、**OAuth 2.0 (3L0)**の横にある**Configure**をクリックして、**Callback URL**の下にコールバックURLを追加してください

![gptactions_jira_callback.png](../../../images/gptactions_jira_callback.png)

### FAQ & トラブルシューティング

- **コールバックURLエラー**: ChatGPTでコールバックURLエラーが発生した場合は、コールバックURLの値を再確認してください。認証に加えられた変更によって、この値が時々変更される可能性があります。

*優先して対応してほしい統合機能はありますか？統合機能にエラーがありますか？GitHubでPRやissueを作成していただければ、確認いたします。*